<a href="https://colab.research.google.com/github/lukekolbe/AL-in-CreditScoring/blob/main/threshold_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
############ LIBRARIES

!pip install scikit-plot


import os
import random
import multiprocessing
import pickle
import copy
import gc
import sys
import json
gc.enable()

import warnings
warnings.filterwarnings('ignore')


import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt

from sklearn.preprocessing import RobustScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold  ##### what is this used for?
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay


############ RANDOMNESS
# seed function
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 30
seed_everything(seed)


In [ ]:
os.chdir('/gdrive/My Drive/ACTIVE LEARNING THESIS/')

# Data Loader

In [ ]:
############ DATA IMPORT

## available datasets
# bene1         # some 

    # OK gmsc          # shape:  (150000, 68)
    # OK uk            # very low bad rate, shape:  (30000, 51), y mean:  0.04
    # OK lendingclub   # loooow performance, X shape:  (41623, 114) y shape:  (41623,) y mean:  0.1331235134420873    
    
    # OK bene2         # some learning, shape:  (7190, 28) >> 3 folds NO WASTE
    
    # bene1_nobins  # shape:  (3123, 18) # 5 folds NO WASTE
        # hmeq          # not promising either, shape:  (5960, 20) # 5 folds NO WASTE
        # australian    # very small dataset shape:  (690, 42) >> 10 folds NO WASTE
        # german        # shape:  (1000, 61) >> 10 folds NO WASTE
        # thomas        # loooow performance shape:  (1225, 28) >> 10 folds NO WASTE

# pakdd         # shape:  (50000, 373), y mean:  0.26082

data_name = "uk"
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "australian", "german", "thomas", "pakdd"]

def data_loader(dataset):

  #df = pd.read_csv('//home//RDC//kolbeluk1//AL_THESIS//prepared_data//{}.csv'.format(dataset)) #Linux path

  # C:\\Users\\kolbeluk1\\AL_THESIS
  #df = pd.read_csv('C:\\Users\\kolbeluk1\\AL_THESIS\\prepared_data\\{}.csv'.format(dataset))
  df = pd.read_csv('/gdrive/My Drive/ACTIVE LEARNING THESIS/prepared_data/{}.csv'.format(dataset))

  # remove NA
  df = df.dropna()
  df.reset_index(drop = True, inplace = True)

  #print(df)
  # extract label
  df['BAD'][df['BAD']=='BAD']  = 1
  df['BAD'][df['BAD']=='GOOD'] = 0
  df['BAD'] = df['BAD'].astype('int')


  y_temp = df['BAD']
  del df['BAD']

  #one hot encoding
  df = pd.get_dummies(df)

  #transform to numpy array >> same location for df and X
  X = df.to_numpy()
  y = y_temp.to_numpy()

  print("X type: ", type(X), "X shape: ",X.shape,"y shape: ", y.shape, "y mean: ", np.mean(y))

  return X,y


# Helper Functions

In [ ]:
# append_record: helper function that adds best-parameter for every model to dict and saves it
def append_record(record, filename):
    with open(f'{filename}', 'a') as f:
        json.dump(record, f)
        f.write(os.linesep)

In [ ]:
# loader function that unpacks tuning results and extracts parameters for different model steps
def param_getter(tuned=False, dataset=None):
  filename = f'{dataset}_tuned-params'

  with open(filename, 'r') as f:
    param_list = [json.loads(line) for line in f if line.startswith('{')]

  param_dict = {}
  for i in range(len(param_list)):
    strategy_short = list(param_list[i].keys())[0]
    param_dict[strategy_short] = param_list[i][list(param_list[i].keys())[0]]

  #find cases where some models are not tuned, establish base parameters
  for key, name in [('oracle', 'Oracle'),
                    ('score', 'Score'),
                    ('eer', 'QueryExpectedErrorReduction'), 
                    ('quire', 'QueryInstanceQUIRE'), 
                    ('bmdr','QueryInstanceBMDR'),
                    ('spal', 'QueryInstanceSPAL')]:
    try:
      param_dict[key]
    except KeyError:
      param_dict[key] = {'AL':{'strategy_name':name}, 'CLF':{}}

  # transfer tuned classifier to all models (clf is tuned separately, not in combination with AL model)
  for key in param_dict.keys():
    param_dict[key]['CLF']=param_dict['random']['CLF']

  for key in ['bmdr', 'spal']:
    param_dict[key]['AL']['rho'] = 10 #set this parameter for increased performance
  
  return param_dict

In [ ]:
'''
  applies every threshold to a given prediction vector and returns misclassification cost as defined in the cost matrix
'''

def cost_tuner(y, pred_raw, y_mean, thres_array, cost_matrix = None):
  if cost_matrix == None:
    fn_cost = (1-y_mean)/y_mean # set fn cost to the inverse of the probability of the rare class; fix cost of fp to 1
    cost_matrix = [[0, 1],[fn_cost, 0]]

  print(cost_matrix)

  cost_list = []
  for t in thres_array:
    pred_thres =  (pred_raw[:,1] >= t).astype(int)
    if thres_array.index(t)%100 == 0:
      print(f'from cost tuner: confusion mat for threshold {t}:',confusion_matrix(y, pred_thres).ravel())
      print(f'from cost tuner: cost for threshold {t}:',np.array([confusion_matrix(y, pred_thres)* cost_matrix]).ravel())
    cost = np.sum(confusion_matrix(y, pred_thres) * cost_matrix)
    cost_list.append(cost)

  print(cost_list)
  cost_min_fold = min(cost_list)
  print('cost_min_fold', cost_min_fold)
  cost_min_fold_index = cost_list.index(cost_min_fold)
  print('cost_min_fold_index', cost_min_fold_index)
  best_threshold_fold = thres_array[cost_min_fold_index]
  print(f'from cost tuner: best threshold of this fold: {best_threshold_fold}; minimum cost: {cost_min_fold}')

  return cost_list

In [ ]:
'''
  the pipeline that loads pre-tuned classifier parameters, processes data (scaling, splitting), 
  trains the tuned classifier, makes a prediction, 
  computes cost for all thresholds, picks threshold that minimizes cost
'''

def threshold_finder(X,y,dataset,folds,cost_matrix):
  seed_everything(seed)

  param_dict = param_getter(tuned=True, dataset=dataset) # load tuned parameters
  clf = LogisticRegression(random_state = seed,**param_dict['random']['CLF']) #set up tuned classifier
  
  skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state=seed)
  idx = []
  for train_index, test_index in skf.split(X, y):
    idx.append((train_index, test_index))

  cost_list = []
  thres_arr =  [np.round(i, 3) for i in iter(np.linspace(0, 1, 1001))] # 1000 possible thresholds between 0 and 1

  y_mean = np.mean(y)


  for f in range(folds-1):
    if len(y)<2000:
      train_idx = idx[f][0]
      test_idx = idx[f][1]
    else:
      #for large datasets, swap order of train and test indices in order to not train on too much data and closer reflect the conditions within the experiment
      train_idx = idx[f][1]
      test_idx = idx[f][0]
    
    if f == 0:
      print('lenghts of train & test: ',len(train_idx), len(test_idx))


    scaler = RobustScaler(with_centering=True, with_scaling=True)
    scaler.fit(X[train_idx,:])
    X_t = scaler.transform(np.array(X)) #scaled version of full dataset 

    clf.fit(X_t[train_idx,:], y[train_idx])
    probabilities = clf.predict_proba(X_t[test_idx,:])

    # for each threshold, compute the misclassification cost and add it to the list
    cost_list.append(cost_tuner(y[test_idx], probabilities, y_mean, thres_arr, cost_matrix = cost_matrix))

    print(np.shape(cost_list))
  
  #compute mean cost for each threshold across training folds
  mean_cost = np.mean(cost_list, axis = 0)
  print(mean_cost)
  cost_min = min(mean_cost)
  cost_min_index = np.where(mean_cost == mean_cost.min())[0][0]
  best_thres = np.round(thres_arr[cost_min_index], 3)

  return best_thres



# Run for all datasets in one go

In [ ]:
threshold_dict = {}
folds = 5
cost_matrices = [None] #other cost matrices not feasible


for cost_matrix in cost_matrices:
  for d in dataset_list:
    X,y = data_loader(d)

    filename = 'tuned_thresholds'
    
    if cost_matrix != None:
      filename += f"_cost-{cost_matrix[0][1]}-{cost_matrix[1][0]}"

    cost_threshold = threshold_finder(X, y, d, folds, cost_matrix = cost_matrix)
    print(f'for dataset {d}, and cost matrix {cost_matrix}, best threshold is {cost_threshold}')
    print("---------------------------------------------")

    threshold_dict[f'{d}'] = copy.deepcopy(cost_threshold)

  print(threshold_dict)

  with open(f'{filename}', 'wb') as a_file:
    pickle.dump(threshold_dict, a_file)

  a_file.close()

X type:  <class 'numpy.ndarray'> X shape:  (7190, 28) y shape:  (7190,) y mean:  0.3
lenghts of train & test:  1438 5752
[[0, 1], [2.3333333333333335, 0]]
from cost tuner: confusion mat for threshold 0.0: [   0 4027    0 1725]
from cost tuner: cost for threshold 0.0: [   0. 4027.    0.    0.]
from cost tuner: confusion mat for threshold 0.1: [ 858 3169   33 1692]
from cost tuner: cost for threshold 0.1: [   0. 3169.   77.    0.]
from cost tuner: confusion mat for threshold 0.2: [1661 2366  129 1596]
from cost tuner: cost for threshold 0.2: [   0. 2366.  301.    0.]
from cost tuner: confusion mat for threshold 0.3: [2441 1586  359 1366]
from cost tuner: cost for threshold 0.3: [   0.         1586.          837.66666667    0.        ]
from cost tuner: confusion mat for threshold 0.4: [3117  910  684 1041]
from cost tuner: cost for threshold 0.4: [   0.  910. 1596.    0.]
from cost tuner: confusion mat for threshold 0.5: [3598  429 1116  609]
from cost tuner: cost for threshold 0.5: [   0

In [ ]:
fname = "tuned_thresholds_new"
infile = open(f'{fname}','rb')
tuned_thresholds_none = pickle.load(infile)
infile.close()

print(tuned_thresholds_none)

{'bene2': 0.283, 'bene1_nobins': 0.372, 'gmsc': 0.057, 'uk': 0.039, 'lendingclub': 0.136, 'hmeq': 0.208, 'australian': 0.333, 'german': 0.336, 'thomas': 0.357, 'pakdd': 0.244}


# RUN FOR SINGLE DATASET

In [ ]:
data_name = "australian" #lendingclub, german, australian

X,y = data_loader(data_name)

X type:  <class 'numpy.ndarray'> X shape:  (690, 42) y shape:  (690,) y mean:  0.4449275362318841


In [ ]:
param_dict = param_getter(tuned=True, dataset = data_name) # load tuned parameters
clf = LogisticRegression(**param_dict['random']['CLF'])
print(clf)

LogisticRegression(C=0.1, max_iter=50, penalty='l1', solver='liblinear',
                   tol=0.001)


In [ ]:
threshold_dict = {}
folds = 5
cost_matrix = None #NONE means cost is set dynamically. The function is able to take in specific cost matrices as put out by sklearn.metrics.confusion_matrix()

filename = 'TEST_tuned_thresholds'
try:
  filename += f"_cost-{cost_matrix[0][1]}-{cost_matrix[1][0]}"
except TypeError:
  filename += f"_cost-None"

for d in ['australian']:
  X,y = data_loader(d)

  cost_threshold = threshold_finder(X, y, d, folds, cost_matrix = cost_matrix)
  print(f'for dataset {d}, best threshold is {cost_threshold}')
  print("---------------------------------------------")

  threshold_dict[f'{d}'] = copy.deepcopy(cost_threshold)

for k in threshold_dict.keys():
  print(threshold_dict[k])


# MANUAL RUN FOR DEBUGGING

In [ ]:
#MANUAL RUN OF OPTIMIZATION SCHEME FOR TESTING, DEBUGGING

folds = 5

#cost_matrix = [[-4,1],[16,-2]] #tn, fp, fn, tp
cost_matrix=None

# SPLIT DATA
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state=seed)
idx = []
for train_index, test_index in skf.split(X, y):
  idx.append((train_index, test_index))

print(np.shape(idx))
cost_list = []
thres_arr =  [np.round(i,3) for i in iter(np.linspace(0, 1, 101))]


y_mean = np.mean(y)
print(f'y mean of whole data: ', y_mean)
print('DEFAULT FN cost: ',(1-y_mean)/y_mean)

print('no. of training samples: ',len(idx[0][1]))

for f in range(folds-1):
  train_idx = idx[f][1] #swap order: use many small training sets and large test sets >> closer to conditions in experiment, where small samples are used...
  test_idx = idx[f][0]
  print('lenghts of train & test: ',len(train_idx), len(test_idx))

  scaler = RobustScaler(with_centering=True, with_scaling=True)
  scaler.fit(X[train_idx,:])
  X_t = scaler.transform(np.array(X)) #scaled version of full dataset

  clf.fit(X_t[train_idx,:], y[train_idx])
  probabilities = clf.predict_proba(X_t[test_idx,:])

  print('mean of prediction: ', np.mean(probabilities[:,1]))
  print('median of prediction: ', np.median(probabilities[:,1]))

  sns.set_style('whitegrid')
  sns.kdeplot(probabilities[:,1]).set_title('Density Plot of outside test prediction')
  plt.show()

  cost_list.append(cost_tuner(y[test_idx], probabilities, y_mean, thres_arr, cost_matrix=cost_matrix))

  print("############## END OF LOOP ################")

print(cost_list)
#print('len cost list ', len(cost_list))

mean_cost = np.mean(cost_list, axis = 0)
print('mean cost: ', mean_cost)

cost_min = min(mean_cost)
print('minimum cost: ', cost_min)
cost_min_index = np.where(mean_cost == mean_cost.min())[0][0]

best_thres = thres_arr[cost_min_index]
print("best threshold: ",best_thres)


train_idx = idx[folds-1][1] #swap order: use many small training sets and large test sets >> closer to conditions in experiment, where small samples are used...
test_idx = idx[folds-1][0]
clf.fit(X[train_idx,:], y[train_idx])
probabilities = clf.predict_proba(X[test_idx,:])
print('mean of prediction: ', np.mean(probabilities[:,1]))
print('median of prediction: ', np.median(probabilities[:,1]))

skplt.metrics.plot_calibration_curve(y[test_idx],[probabilities], ['LR'], title = 'Calibration plot of outside test prediction')
plt.show()

sns.set_style('whitegrid')
sns.kdeplot(probabilities[:,1]).set_title('Density Plot of outside test prediction')
plt.show()
    

pred_thres = (probabilities[:,1] >= best_thres).astype(int)
#pred_thres = (probabilities[:,1] >= 0.5).astype(int)
print('mean of thresholded prediction: ', np.mean(pred_thres))
print(probabilities[:20,1])
print(pred_thres[:15])
print('percentage kept: ',1-np.mean(pred_thres))

disp = ConfusionMatrixDisplay.from_predictions(
    y[test_idx],
    pred_thres,
)
print(disp.confusion_matrix)

plt.show()
